# **Ф35**: Модуль прогнозирования эффектов трансформации территории в масштабах населенного пункта и региона

## 1. Подготовка исходных данных

- `service_types` -- таблица типов сервисов.
- `blocks` -- слой городских кварталов с разной информацией.
- `acc_mx` -- матрица доступности городских кварталов.

In [2]:
import pandas as pd
import geopandas as gpd
from blocksnet.config import log_config

log_config.set_disable_tqdm(True)
log_config.set_logger_level('ERROR')

service_types = pd.read_pickle('./data/service_types.pickle')
blocks = pd.read_pickle('./data/blocks.pickle')
acc_mx = pd.read_pickle('./data/acc_mx.pickle')

### 1.1. Подготовка списка сервисов

Нужны только те списки сервисов, у которых есть `infrastructure_type`

In [3]:
service_types = service_types[~service_types['infrastructure_type'].isna()].copy()
service_types.head()

,name,infrastructure_type,infrastructure_weight,social_values
service_type_id,,,,
1,park,basic,0.2,[8]
5,beach,comfort,0.5,None
21,kindergarten,basic,0.2,[2]
22,school,basic,0.2,"[2, 4]"
27,university,comfort,0.2,"[4, 10, 17]"


Также для каждого `infrastructure_type` также добавим весовой коэффициент, чтобы отдать предпочтение базовым сервисам, а остальные добавлять по возможности.

In [4]:
INFRASTRUCTURES_WEIGHTS = {
    'basic': 0.5714,
    'additional': 0.2857,
    'comfort': 0.1429
}

service_types['infrastructure_weight'] = service_types['infrastructure_type'].map(INFRASTRUCTURES_WEIGHTS) * service_types['infrastructure_weight']
service_types.head()

,name,infrastructure_type,infrastructure_weight,social_values
service_type_id,,,,
1,park,basic,0.11428,[8]
5,beach,comfort,0.07145,None
21,kindergarten,basic,0.11428,[2]
22,school,basic,0.11428,"[2, 4]"
27,university,comfort,0.02858,"[4, 10, 17]"


## 2. Эффекты

Тут есть важная проблема: 
- В `blocks` хранится информация о сценарии `user_scenario_id` как есть и результирующих сервисах, населении и функциональных зонах.
- Для того, чтоб узнать как было ДО, нужно отдельно считать обеспеченность по контексту по `base_scenario_id`.

Результат:

- Оценки обеспеченности по каждому возможному `service_type`.
- Застройка, которая уходит в **Ф26**.

### 2.b.1. Подготовка данных

Готовим словарь, какой квартал в какой ФЗ переводим.

`blocks_lus : dict[int, LandUse]`

In [5]:
blocks_lus = blocks.loc[blocks['is_project'],'land_use']
blocks_lus = blocks_lus[~blocks_lus.isna()]
blocks_lus = blocks_lus.to_dict()

Инициализируем экземпляр класса оптимизатора.

In [6]:
from blocksnet.optimization.services import (
    TPEOptimizer,
    WeightedObjective,
    WeightedConstraints,
    Facade,
    BlockSolution,
    GradientChooser,
)
from tqdm import tqdm

var_adapter = BlockSolution(blocks_lus)

facade = Facade(
    blocks_lu=blocks_lus,
    blocks_df=blocks[['site_area', 'population']].fillna(0),
    accessibility_matrix=acc_mx,
    var_adapter=var_adapter,
)

Добавляем информацию о существующих типах сервисов.

**ВАЖНО**: если сервиса нет в слое кварталов, просто добавляем `DataFrame` с нулевыми `capacity` для него. Ведь мы же должны что-то расставлять.

In [7]:
for st_id, row in service_types.iterrows():
    st_name = row['name']
    st_weight = row['infrastructure_weight']
    st_column = f'capacity_{st_name}'
    if st_column in blocks.columns:
        df = blocks.rename(columns={st_column: 'capacity'})[['capacity']].fillna(0)
    else:
        print(f'#{st_id}:{st_name} нет на территории контекста проекта. Добавляем нулевой датафрейм')
        df = blocks[[]].copy()
        df['capacity'] = 0
    facade.add_service_type(st_name, st_weight, df)

#5:beach нет на территории контекста проекта. Добавляем нулевой датафрейм
#51:theatre нет на территории контекста проекта. Добавляем нулевой датафрейм
#56:cinema нет на территории контекста проекта. Добавляем нулевой датафрейм
#143:sanatorium нет на территории контекста проекта. Добавляем нулевой датафрейм


### 2.b.2. Оптимизация

Запускаем оптимизатор...

In [8]:
services_weights = service_types.set_index('name')['infrastructure_weight'].to_dict()

objective = WeightedObjective(num_params=facade.num_params, facade=facade, weights=services_weights, max_evals=50)

constraints = WeightedConstraints(num_params=facade.num_params, facade=facade)

tpe_optimizer = TPEOptimizer(
    objective=objective, constraints=constraints, vars_chooser=GradientChooser(facade, facade.num_params, 5)
)

best_x, best_val, perc, func_evals = tpe_optimizer.run(max_runs=1000, timeout=60000, initial_runs_num=1, verbose=False)

[I 2025-06-02 00:29:26,006] A new study created in memory with name: no-name-34075fa6-d3c9-4f98-b9d3-6c750fc1840d
[I 2025-06-02 00:29:26,877] Trial 0 finished with value: 0.6798072322327586 and parameters: {'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 0, 'x_4': 0, 'x_5': 0, 'x_6': 0, 'x_7': 0, 'x_8': 0, 'x_9': 0, 'x_10': 0, 'x_11': 0, 'x_12': 0, 'x_13': 0, 'x_14': 0, 'x_15': 0, 'x_16': 0, 'x_17': 0, 'x_18': 0, 'x_19': 0, 'x_20': 0, 'x_21': 0, 'x_22': 0, 'x_23': 0, 'x_24': 0, 'x_25': 0, 'x_26': 0, 'x_27': 0, 'x_28': 0, 'x_29': 0, 'x_30': 0, 'x_31': 0, 'x_32': 0, 'x_33': 0, 'x_34': 0, 'x_35': 0, 'x_36': 0, 'x_37': 0, 'x_38': 0, 'x_39': 0, 'x_40': 0, 'x_41': 0, 'x_42': 0, 'x_43': 0, 'x_44': 0, 'x_45': 0, 'x_46': 0, 'x_47': 0, 'x_48': 0, 'x_49': 0, 'x_50': 0, 'x_51': 0, 'x_52': 0, 'x_53': 0, 'x_54': 0, 'x_55': 0, 'x_56': 0, 'x_57': 0, 'x_58': 0, 'x_59': 0, 'x_60': 0, 'x_61': 0, 'x_62': 0, 'x_63': 0, 'x_64': 0, 'x_65': 0, 'x_66': 0, 'x_67': 0, 'x_68': 0, 'x_69': 0, 'x_70': 0, 'x_71': 0, 'x_72': 0, 

### 2.b.3. Результат

In [ ]:
facade.p

{'beach': 0.0,
 'kindergarten': 0.0,
 'school': 0.674074074074074,
 'university': 0.0,
 'polyclinic': 0.0,
 'hospital': 0.5707547169811321,
 'museum': 0.9856115107913669,
 'theatre': 0.0,
 'cinema': 0.0,
 'mall': 0.8875,
 'stadium': 0.19327731092436976,
 'restaurant': 0.6990838618745595,
 'bar': 0.4642857142857143,
 'police': 0.4,
 'train_station': 0.3865546218487395,
 'supermarket': 0.9897209985315712,
 'market': 0.6977599465061852,
 'veterinary': 0.711864406779661,
 'religion': 0.6984924623115578,
 'sanatorium': 0.0}

# **Ф26**: Модуль ценностно-ориентированного моделирования программ развития населенного пункта.

Берем результат из пункта `2.b` и разворачиваем его на кварталы: куда что ставить:

- `block_id` -- номер квартала, куда ставим.
- `service_type` -- тип сервиса.
- `site_area` -- занимаемая площадь квартала.
- `build_floor_area` -- занимаемая площадь зданий.
- `capacity` -- суммарная емкость типа застройки
- `count` -- количество расставляемых объектов данного типа в данном квартале

P.S: сейчас результаты не выглядят правдиво. Оптимизатор дорабатывается, формат останется таким же.

In [39]:
solution_df = facade.solution_to_services_df(best_x)
solution_df

,block_id,service_type,site_area,build_floor_area,capacity,count
0,8,beach,1440.0,0.0,480,2
3,13,beach,1440.0,0.0,480,1
6,14,beach,1440.0,0.0,480,1
9,15,beach,1440.0,0.0,480,1
15,36,beach,1440.0,0.0,480,1
...,...,...,...,...,...,...
1668,67,train_station,400.0,0.0,20,3
1669,67,train_station,650.0,0.0,40,2
1670,67,train_station,1300.0,0.0,100,1
2193,1,sanatorium,15000.0,3000.0,100,1
